In [1]:
# -*- coding: utf-8 -*-
# ============================================================
# SKRIP EKSTRAKSI FITUR MANUAL EMG + EXPORT KE CSV
# - Time-domain (7 fitur per channel)
# - Frequency-domain (6 fitur per channel)
# - Magnitude-squared coherence (MSC, 1 fitur per pasangan channel)
# ============================================================
import numpy as np
import pandas as pd
from scipy import signal
import os
import zipfile
import io

# ------------------------------------------------------------
# MSC (Magnitude-Squared Coherence) antar channel
# ------------------------------------------------------------
def compute_msc(emg_data, fs=1000, nperseg=500):
    """ 
    emg_data: array (n_samples, n_timesteps, n_channels)
    return: (n_samples, n_pairs) -> rata-rata coherence antar pasangan channel 
    """
    emg_data = np.asarray(emg_data)
    n_samples, n_timesteps, n_channels = emg_data.shape
    
    # Hitung jumlah pasangan channel: n * (n - 1) / 2
    n_pairs = n_channels * (n_channels - 1) // 2
    msc_features = np.zeros((n_samples, n_pairs))

    for i in range(n_samples):
        pair_idx = 0
        for ch1 in range(n_channels):
            for ch2 in range(ch1 + 1, n_channels):
                # Hitung Magnitude-Squared Coherence
                # Mengambil rata-rata Cxy sebagai 1 fitur MSC per pasangan
                f, Cxy = signal.coherence(
                    emg_data[i, :, ch1], 
                    emg_data[i, :, ch2], 
                    fs=fs, 
                    nperseg=nperseg
                )
                msc_features[i, pair_idx] = np.mean(Cxy)
                pair_idx += 1
    return msc_features

# ------------------------------------------------------------
# TIME-DOMAIN FEATURES (7 fitur per channel)
# ------------------------------------------------------------
def extract_time_domain_features(emg_data):
    """ 
    emg_data: (n_samples, n_timesteps, n_channels)
    Fitur per channel: MAV, RMS, VAR, SSI, ZC, SSC, WL
    Total fitur = n_channels * 7
    """
    emg_data = np.asarray(emg_data)
    n_samples, n_timesteps, n_channels = emg_data.shape
    n_feats_per_ch = 7
    features = np.zeros((n_samples, n_channels * n_feats_per_ch))

    for i in range(n_samples):
        for ch in range(n_channels):
            sig = emg_data[i, :, ch]
            
            # 0. Mean Absolute Value (MAV)
            mav = np.mean(np.abs(sig))
            # 1. Root Mean Square (RMS)
            rms = np.sqrt(np.mean(sig ** 2))
            # 2. Variance (VAR)
            var = np.var(sig)
            # 3. Simple Square Integral (SSI)
            ssi = np.sum(sig ** 2)
            
            # 4. Zero Crossings (ZC) (Normalized)
            zc = np.sum(np.diff(np.sign(sig))!= 0) / n_timesteps
            
            # 5. Slope Sign Changes (SSC) (Normalized)
            diff_sig = np.diff(sig)
            ssc = np.sum((diff_sig[:-1] * diff_sig[1:]) < 0) / n_timesteps
            
            # 6. Waveform Length (WL)
            wl = np.sum(np.abs(np.diff(sig)))

            base = ch * n_feats_per_ch
            features[i, base + 0] = mav
            features[i, base + 1] = rms
            features[i, base + 2] = var
            features[i, base + 3] = ssi
            features[i, base + 4] = zc
            features[i, base + 5] = ssc
            features[i, base + 6] = wl
            
    return features

# ------------------------------------------------------------
# FREQUENCY-DOMAIN FEATURES (6 fitur per channel)
# ------------------------------------------------------------
def extract_frequency_domain_features(emg_data, fs=1000):
    """ 
    emg_data: (n_samples, n_timesteps, n_channels)
    Fitur per channel: MNF, MDF, PKF, MNP, TTP, BPr (Band power ratio 10-50 Hz)
    Total fitur = n_channels * 6
    """
    emg_data = np.asarray(emg_data)
    n_samples, n_timesteps, n_channels = emg_data.shape
    n_feats_per_ch = 6
    features = np.zeros((n_samples, n_channels * n_feats_per_ch))
    
    for i in range(n_samples):
        for ch in range(n_channels):
            sig = emg_data[i, :, ch]
            # Hitung power spectral density (PSD)
            f, Pxx = signal.welch(sig, fs=fs, nperseg=min(256, n_timesteps))
            
            if np.sum(Pxx) == 0:
                mnf, mdf, pkf, mnp, ttp, bpr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            else:
                total_power = np.sum(Pxx)
                
                # 0. Mean frequency (MNF)
                mnf = np.sum(f * Pxx) / total_power
                
                # 1. Median frequency (MDF)
                cumsum = np.cumsum(Pxx)
                median_idx = np.where(cumsum >= total_power / 2)[0][0]
                mdf = f[median_idx]
                
                # 2. Peak Frequency (PKF)
                peak_idx = np.argmax(Pxx)
                pkf = f[peak_idx]
                
                # 3. Mean Power (MNP)
                mnp = np.mean(Pxx)
                
                # 4. Total Power (TTP)
                ttp = total_power
                
                # 5. Band power ratio (BPr) 10-50 Hz
                mask = (f >= 10) & (f <= 50)
                band_power = np.sum(Pxx[mask])
                bpr = band_power / ttp if ttp > 0 else 0.0
            
            base = ch * n_feats_per_ch
            features[i, base + 0] = mnf
            features[i, base + 1] = mdf
            features[i, base + 2] = pkf
            features[i, base + 3] = mnp
            features[i, base + 4] = ttp
            features[i, base + 5] = bpr
            
    return features

# ------------------------------------------------------------
# PREPARE DATA DENGAN META (untuk windowing)
# ------------------------------------------------------------
def prepare_data_with_meta(df, channels, label_col='group', meta_cols=None, window_size=1000, step_size=1000):
    """ 
    Membagi sinyal menjadi window-window dengan metadata.
    df: DataFrame yang kolom sinyalnya berisi list/array sinyal.
    return: X (Windows), y (Labels), meta_list (Metadata per window)
    """
    if meta_cols is None:
        meta_cols = []

    X, y = [], []
    meta_list = []
    n_timesteps = window_size

    for _, row in df.iterrows():
        signals = [row[ch] for ch in channels]
        sig_arr = np.array([np.array(sig) for sig in signals])

        if any(len(sig) == 0 for sig in sig_arr):
            continue
            
        min_len = min(len(sig) for sig in sig_arr)
        
        for start in range(0, min_len - n_timesteps + 1, step_size):
            window = np.array([sig[start:start+n_timesteps] for sig in sig_arr])
            X.append(window.T) # (time, channels)
            y.append(row[label_col])
            
            # Simpan metadata
            meta = {col: row.get(col) for col in meta_cols}
            meta_list.append(meta)

    return np.array(X), np.array(y), meta_list

# ------------------------------------------------------------
# FUNGSI UTAMA EKSTRAKSI + SIMPAN CSV
# ------------------------------------------------------------
def extract_and_save_emg_features(df, channels, fs=1000, label_col='group', meta_cols=None, window_size=1000, step_size=1000, out_csv='emg_features.csv'):
    """ 
    Ekstrak fitur EMG manual dan simpan ke CSV.
    """
    
    # 1. Prepare data (Windowing + Meta)
    print("[INFO] Membentuk window EMG dan mengumpulkan metadata...")
    X, y, meta_list = prepare_data_with_meta(
        df, channels, label_col=label_col, meta_cols=meta_cols, 
        window_size=window_size, step_size=step_size
    )
    
    n_windows, n_timesteps, n_channels = X.shape
    print(f"[INFO] Total windows: {n_windows}, Time steps: {n_timesteps}, Channels: {n_channels}")

    # 2. Ekstraksi Fitur
    print("[INFO] Ekstraksi fitur time-domain...")
    time_feats = extract_time_domain_features(X)
    
    print("[INFO] Ekstraksi fitur frequency-domain...")
    freq_feats = extract_frequency_domain_features(X, fs=fs)
    
    print("[INFO] Ekstraksi fitur MSC...")
    msc_feats = compute_msc(X, fs=fs, nperseg=500)
    
    print("[INFO] Menggabungkan semua fitur...")
    all_features = np.concatenate([time_feats, freq_feats, msc_feats], axis=1)
    n_total_feats = all_features.shape[1]
    print(f"[INFO] Dimensi fitur akhir: {n_windows} windows x {n_total_feats} fitur")

    # 3. Bangun DataFrame Hasil
    rows = []
    
    time_feat_names = [f'time_f{j+1}' for j in range(time_feats.shape[1])]
    freq_feat_names = [f'freq_f{j+1}' for j in range(freq_feats.shape[1])]
    msc_feat_names = [f'msc_f{j+1}' for j in range(msc_feats.shape[1])]
    
    feature_names = time_feat_names + freq_feat_names + msc_feat_names
    
    for i in range(n_windows):
        row_dict = {}
        
        # Masukkan meta dan label
        if meta_cols is not None:
            row_dict.update(meta_list[i])
        row_dict[label_col] = y[i]
        
        # Masukkan fitur
        for k, name in enumerate(feature_names):
            row_dict[name] = all_features[i, k]
            
        rows.append(row_dict)

    df_features = pd.DataFrame(rows)

    # 4. Simpan ke CSV
    # Pembulatan numerik ke 5 desimal
    num_cols = df_features.select_dtypes(include=[float, np.number]).columns
    df_features[num_cols] = df_features[num_cols].round(5)
    
    df_features.to_csv(out_csv, index=False)
    print(f"[INFO] Fitur EMG berhasil tersimpan ke: {out_csv}")
    
    return df_features

# ============================================================================
# CONTOH PENGGUNAAN / SIMULASI DATA (HARAP DISESUAIKAN)
# ============================================================================

# --- FUNGSI MOCK UNTUK DATA LOADING DARI FILE (sesuaikan dengan data Anda) ---

def process_emg_file(filepath):
    """MOCK: Ganti dengan logic membaca sinyal dari file CSV Anda"""
    # Karena data Anda adalah list/array sinyal 1D di tiap kolom DataFrame
    # Fungsi ini tidak diperlukan jika DataFrame sudah dimuat.
    # Jika perlu memproses file, pastikan membaca dua kolom (right, left)
    if os.path.exists(filepath):
        # Contoh: membaca dari file dengan 2 kolom sinyal
        df = pd.read_csv(filepath, header=None)
        return df.iloc[:, 0].dropna().values, df.iloc[:, 1].dropna().values
    return np.array([]), np.array([])


def create_mock_dataframe():
    """
    Membuat DataFrame MOCK yang meniru struktur yang diharapkan: 
    setiap baris adalah 1 percobaan/subjek, kolom sinyal berisi list/array sinyal 1D.
    """
    np.random.seed(42)
    n_subjects = 20
    signal_length = 20000 
    
    # Definisikan 8 Channel EMG
    channels = [
        'RightLatissimusDorsi', 'LeftLatissimusDorsi', 
        'RightC4Paraspinal', 'LeftC4Paraspinal', 
        'RightSternocleidomastoidcaputlateralis', 'LeftSternocleidomastoidcaputlateralis', 
        'RightTrapeziusdescendens', 'LeftTrapeziusdescendens'
    ]
    
    data = []
    for i in range(n_subjects):
        group = 1 if i % 2 == 0 else 0 # 10 CNP, 10 CONT
        condition = 'Curv_CCW' if i < 10 else 'Rect_RECT'
        
        row = {
            'subject_number': i + 1,
            'group': group,
            'condition': condition
        }
        
        # Isi sinyal (mock data)
        for ch in channels:
            # Sinyal + sedikit tren (simulasi EMG)
            noise = np.random.normal(0, 0.01 + 0.001 * group, signal_length)
            trend = np.linspace(0, 0.1, signal_length) * (1 - group) 
            raw_signal = noise + trend 
            row[ch] = raw_signal.tolist()
            
        data.append(row)
        
    df_mock = pd.DataFrame(data)
    # Map label 'group' ke 0/1 jika belum
    df_mock['group'] = df_mock['group'].astype(int) 
    
    return df_mock, channels

# --- BLOK EKSEKUSI ---
if __name__ == '__main__':
    
    # 1. Buat data mock (GANTI DENGAN DATA LOADING ASLI ANDA)
    df_raw, channels = create_mock_dataframe() 
    
    # Contoh untuk kondisi Curvilinear saja
    df_curvilinear = df_raw[df_raw['condition'].str.contains('Curv')].reset_index(drop=True)

    print(f"Dataframe siap. Channels: {channels}")
    print(f"Total baris (percobaan): {len(df_curvilinear)}")

    # 2. Atur parameter ekstraksi
    WINDOW_SIZE = 1000  # 1 detik @ @1000 Hz
    STEP_SIZE = 500     # overlap 50%
    FS = 1000
    
    # Meta yang ingin dimasukkan ke CSV hasil
    META_COLS = ['subject_number', 'condition'] 
    LABEL_COL = 'group' 
    OUTPUT_CSV = 'emg_features_curvilinear_extracted.csv'

    # 3. Jalankan ekstraksi dan simpan CSV
    df_features = extract_and_save_emg_features(
        df=df_curvilinear,
        channels=channels,
        fs=FS,
        label_col=LABEL_COL,
        meta_cols=META_COLS,
        window_size=WINDOW_SIZE,
        step_size=STEP_SIZE,
        out_csv=OUTPUT_CSV
    )
    
    # Tampilkan 5 baris pertama hasil
    print("\n[INFO] Preview Hasil Fitur:")
    print(df_features.head())



Dataframe siap. Channels: ['RightLatissimusDorsi', 'LeftLatissimusDorsi', 'RightC4Paraspinal', 'LeftC4Paraspinal', 'RightSternocleidomastoidcaputlateralis', 'LeftSternocleidomastoidcaputlateralis', 'RightTrapeziusdescendens', 'LeftTrapeziusdescendens']
Total baris (percobaan): 10
[INFO] Membentuk window EMG dan mengumpulkan metadata...
[INFO] Total windows: 390, Time steps: 1000, Channels: 8
[INFO] Ekstraksi fitur time-domain...
[INFO] Ekstraksi fitur frequency-domain...
[INFO] Ekstraksi fitur MSC...
[INFO] Menggabungkan semua fitur...
[INFO] Dimensi fitur akhir: 390 windows x 132 fitur
[INFO] Fitur EMG berhasil tersimpan ke: emg_features_curvilinear_extracted.csv

[INFO] Preview Hasil Fitur:
   subject_number condition  group  time_f1  time_f2  time_f3  time_f4  \
0               1  Curv_CCW      1  0.00857  0.01077  0.00012  0.11595   
1               1  Curv_CCW      1  0.00874  0.01096  0.00012  0.12015   
2               1  Curv_CCW      1  0.00871  0.01099  0.00012  0.12087   
3 